# Performing Sentiment Analysis via Textblob and VADER

In [5]:
import re
import string
from os.path import dirname, join, realpath

import nltk
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob

In [6]:
def is_interactive():
    """Check if the script is being run interactively."""
    import __main__ as main

    return not hasattr(main, "__file__")


if is_interactive():
    SCRIPT_DIR = dirname(realpath("__file__"))
else:
    SCRIPT_DIR = dirname(realpath(__file__))

DATA_DIR = join(dirname(SCRIPT_DIR), "data", "processed", "twitter")

In [7]:
data = pd.read_csv(join(DATA_DIR, "tweets_2022_03_05-2022_03_11.csv"))
data.head()

,Unnamed: 0.1,Unnamed: 0,text,retweet_count,favorite_count,followers_count,verified,listed_count,created_at,hashtags,name
0,0,0,"""It is an open ledger, trying to sneak lots of...",151,520,2437101,True,10788,2022-03-05 09:33:07+00:00,"[{'text': 'crypto', 'indices': [61, 68]}]",Bitcoin News
1,1,1,“The #crypto market today has a market capital...,87,245,2437101,True,10788,2022-03-05 12:03:14+00:00,"[{'text': 'crypto', 'indices': [5, 12]}]",Bitcoin News
2,2,2,G7 countries and the EU are looking at ways to...,95,245,2437101,True,10788,2022-03-05 13:33:29+00:00,"[{'text': 'cryptocurrency', 'indices': [88, 10...",Bitcoin News
3,3,3,JUST IN: 🇸🇬 Singapore has introduced sanctions...,149,1043,708057,True,1449,2022-03-05 11:01:13+00:00,"[{'text': 'cryptocurrency', 'indices': [77, 92]}]",Watcher.Guru
4,4,4,Make sure you check in on your bros in the str...,45,149,7822,True,80,2022-03-05 14:45:25+00:00,[],Ian Heinisch


## Clean Tweets

In [8]:
text_data = data[
    [
        "Unnamed: 0",
        "name",
        "verified",
        "created_at",
        "retweet_count",
        "favorite_count",
        "followers_count",
        "hashtags",
        "text",
    ]
]
text_data = text_data.rename(columns={"Unnamed: 0": "tweet_id"})
text_data.head()

,tweet_id,name,verified,created_at,retweet_count,favorite_count,followers_count,hashtags,text
0,0,Bitcoin News,True,2022-03-05 09:33:07+00:00,151,520,2437101,"[{'text': 'crypto', 'indices': [61, 68]}]","""It is an open ledger, trying to sneak lots of..."
1,1,Bitcoin News,True,2022-03-05 12:03:14+00:00,87,245,2437101,"[{'text': 'crypto', 'indices': [5, 12]}]",“The #crypto market today has a market capital...
2,2,Bitcoin News,True,2022-03-05 13:33:29+00:00,95,245,2437101,"[{'text': 'cryptocurrency', 'indices': [88, 10...",G7 countries and the EU are looking at ways to...
3,3,Watcher.Guru,True,2022-03-05 11:01:13+00:00,149,1043,708057,"[{'text': 'cryptocurrency', 'indices': [77, 92]}]",JUST IN: 🇸🇬 Singapore has introduced sanctions...
4,4,Ian Heinisch,True,2022-03-05 14:45:25+00:00,45,149,7822,[],Make sure you check in on your bros in the str...


In [9]:
punctuation = string.punctuation
punctuation = punctuation.replace("#", "")
punctuation = punctuation.replace("@", "")


def remove_punctuation(text):
    punctuationfree = "".join([i for i in text if i not in punctuation])
    return punctuationfree

In [10]:
def tokenization(text):
    tokens = re.split("W+", text)
    return tokens

In [11]:
nltk.download("stopwords")
stopwords = nltk.corpus.stopwords.words("english")


def remove_stopwords(text):
    output = [i for i in text if i not in stopwords]
    return output

[nltk_data] Downloading package stopwords to /home/bryan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
nltk.download("wordnet")
wordnet_lemmatizer = WordNetLemmatizer()


def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text

[nltk_data] Downloading package wordnet to /home/bryan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [13]:
def clean_text(text):
    text["clean_text"] = text["text"].apply(lambda x: remove_punctuation(x))
    text["clean_text"] = text["clean_text"].apply(lambda x: x.lower())
    text["clean_text"] = text["clean_text"].apply(lambda x: tokenization(x))
    text["clean_text"] = text["clean_text"].apply(
        lambda x: remove_stopwords(x)
    )
    text["clean_text"] = text["clean_text"].apply(lambda x: lemmatizer(x))
    clean = text
    return clean

In [14]:
text_data = clean_text(text_data)
text_data.head()

,tweet_id,name,verified,created_at,retweet_count,favorite_count,followers_count,hashtags,text,clean_text
0,0,Bitcoin News,True,2022-03-05 09:33:07+00:00,151,520,2437101,"[{'text': 'crypto', 'indices': [61, 68]}]","""It is an open ledger, trying to sneak lots of...",[it is an open ledger trying to sneak lots of ...
1,1,Bitcoin News,True,2022-03-05 12:03:14+00:00,87,245,2437101,"[{'text': 'crypto', 'indices': [5, 12]}]",“The #crypto market today has a market capital...,[“the #crypto market today has a market capita...
2,2,Bitcoin News,True,2022-03-05 13:33:29+00:00,95,245,2437101,"[{'text': 'cryptocurrency', 'indices': [88, 10...",G7 countries and the EU are looking at ways to...,[g7 countries and the eu are looking at ways t...
3,3,Watcher.Guru,True,2022-03-05 11:01:13+00:00,149,1043,708057,"[{'text': 'cryptocurrency', 'indices': [77, 92]}]",JUST IN: 🇸🇬 Singapore has introduced sanctions...,[just in 🇸🇬 singapore has introduced sanctions...
4,4,Ian Heinisch,True,2022-03-05 14:45:25+00:00,45,149,7822,[],Make sure you check in on your bros in the str...,[make sure you check in on your bros in the st...


In [15]:
text_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3978 entries, 0 to 3977
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   tweet_id         3978 non-null   int64 
 1   name             3978 non-null   object
 2   verified         3978 non-null   bool  
 3   created_at       3978 non-null   object
 4   retweet_count    3978 non-null   int64 
 5   favorite_count   3978 non-null   int64 
 6   followers_count  3978 non-null   int64 
 7   hashtags         3978 non-null   object
 8   text             3978 non-null   object
 9   clean_text       3978 non-null   object
dtypes: bool(1), int64(4), object(5)
memory usage: 283.7+ KB


## Sentiment Analysis

In [16]:
def textblob_polarity(text):
    return TextBlob(text).sentiment.polarity

In [17]:
text_data["textblob_sentiment"] = (
    text_data["clean_text"].astype("str").apply(textblob_polarity)
)

In [18]:
def textblob_sentiment_summary(text):
    sentiment = "Neutral"
    if text > 0:
        sentiment = "Positive"
    elif text < 0:
        sentiment = "Negative"
    return sentiment

In [19]:
text_data["textblob_summary"] = text_data["textblob_sentiment"].apply(
    lambda x: textblob_sentiment_summary(x)
)

In [20]:
text_data.head()

,tweet_id,name,verified,created_at,retweet_count,favorite_count,followers_count,hashtags,text,clean_text,textblob_sentiment,textblob_summary
0,0,Bitcoin News,True,2022-03-05 09:33:07+00:00,151,520,2437101,"[{'text': 'crypto', 'indices': [61, 68]}]","""It is an open ledger, trying to sneak lots of...",[it is an open ledger trying to sneak lots of ...,0.25,Positive
1,1,Bitcoin News,True,2022-03-05 12:03:14+00:00,87,245,2437101,"[{'text': 'crypto', 'indices': [5, 12]}]",“The #crypto market today has a market capital...,[“the #crypto market today has a market capita...,-0.20,Negative
2,2,Bitcoin News,True,2022-03-05 13:33:29+00:00,95,245,2437101,"[{'text': 'cryptocurrency', 'indices': [88, 10...",G7 countries and the EU are looking at ways to...,[g7 countries and the eu are looking at ways t...,0.00,Neutral
3,3,Watcher.Guru,True,2022-03-05 11:01:13+00:00,149,1043,708057,"[{'text': 'cryptocurrency', 'indices': [77, 92]}]",JUST IN: 🇸🇬 Singapore has introduced sanctions...,[just in 🇸🇬 singapore has introduced sanctions...,0.00,Neutral
4,4,Ian Heinisch,True,2022-03-05 14:45:25+00:00,45,149,7822,[],Make sure you check in on your bros in the str...,[make sure you check in on your bros in the st...,0.50,Positive


In [21]:
nltk.download("vader_lexicon")
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/bryan/nltk_data...


In [22]:
text_data["vader_sentiment_pos"] = (
    text_data["clean_text"].astype("str")
).apply(lambda x: sid.polarity_scores(x)["pos"])
text_data["vader_sentiment_neg"] = (
    text_data["clean_text"].astype("str")
).apply(lambda x: sid.polarity_scores(x)["neg"])
text_data["vader_sentiment_neu"] = (
    text_data["clean_text"].astype("str")
).apply(lambda x: sid.polarity_scores(x)["neu"])
text_data["vader_sentiment_compound"] = (
    text_data["clean_text"].astype("str")
).apply(lambda x: sid.polarity_scores(x)["compound"])

In [23]:
def vader_sentiment_summary(text):
    sentiment = "Neutral"
    if text >= 0.05:
        sentiment = "Positive"
    elif text <= -0.05:
        sentiment = "Negative"
    return sentiment

In [24]:
text_data["vader_summary"] = text_data["vader_sentiment_compound"].apply(
    lambda x: vader_sentiment_summary(x)
)

In [25]:
text_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3978 entries, 0 to 3977
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   tweet_id                  3978 non-null   int64  
 1   name                      3978 non-null   object 
 2   verified                  3978 non-null   bool   
 3   created_at                3978 non-null   object 
 4   retweet_count             3978 non-null   int64  
 5   favorite_count            3978 non-null   int64  
 6   followers_count           3978 non-null   int64  
 7   hashtags                  3978 non-null   object 
 8   text                      3978 non-null   object 
 9   clean_text                3978 non-null   object 
 10  textblob_sentiment        3978 non-null   float64
 11  textblob_summary          3978 non-null   object 
 12  vader_sentiment_pos       3978 non-null   float64
 13  vader_sentiment_neg       3978 non-null   float64
 14  vader_se

## Saving Analysis per Coin

In [26]:
bitcoin_sentiment = text_data[text_data["text"].str.contains("bitcoin|BTC")]
ethereum_sentiment = text_data[text_data["text"].str.contains("ethereum|ETH")]
doge_sentiment = text_data[text_data["text"].str.contains("dogecoin|DOGE")]
avalanche_sentiment = text_data[
    text_data["text"].str.contains("avalanche|AVAX")
]
solana_sentiment = text_data[text_data["text"].str.contains("solana|SOL")]
crypto_sentiment = text_data[
    text_data["text"].str.contains("crypto|cryptocurrency|coin")
]

In [27]:
bitcoin_sentiment.to_csv(join(DATA_DIR, "bitcoin_sentiment.csv"))
bitcoin_sentiment.head()

,tweet_id,name,verified,created_at,retweet_count,favorite_count,followers_count,hashtags,text,clean_text,textblob_sentiment,textblob_summary,vader_sentiment_pos,vader_sentiment_neg,vader_sentiment_neu,vader_sentiment_compound,vader_summary
15,9,The Moon,True,2022-03-05 16:55:01+00:00,266,1776,1013020,"[{'text': 'crypto', 'indices': [12, 19]}]",Choose your #crypto team:\n\nTeam 🅰️ ...,[choose your #crypto team\n\nteam 🅰️ ...,0.0,Neutral,0.0,0.0,1.0,0.0,Neutral
23,17,Bitcoin News,True,2022-03-05 16:34:04+00:00,90,267,2437101,"[{'text': 'bitcoin', 'indices': [81, 89]}, {'t...",Shake Shack says participants will get 15% of ...,[shake shack says participants will get 15 of ...,0.0,Neutral,0.0,0.0,1.0,0.0,Neutral
34,9,The Moon,True,2022-03-05 16:55:01+00:00,266,1776,1013020,"[{'text': 'crypto', 'indices': [12, 19]}]",Choose your #crypto team:\n\nTeam 🅰️ ...,[choose your #crypto team\n\nteam 🅰️ ...,0.0,Neutral,0.0,0.0,1.0,0.0,Neutral
42,17,Bitcoin News,True,2022-03-05 16:34:04+00:00,90,267,2437101,"[{'text': 'bitcoin', 'indices': [81, 89]}, {'t...",Shake Shack says participants will get 15% of ...,[shake shack says participants will get 15 of ...,0.0,Neutral,0.0,0.0,1.0,0.0,Neutral
57,9,The Moon,True,2022-03-05 16:55:01+00:00,266,1776,1013020,"[{'text': 'crypto', 'indices': [12, 19]}]",Choose your #crypto team:\n\nTeam 🅰️ ...,[choose your #crypto team\n\nteam 🅰️ ...,0.0,Neutral,0.0,0.0,1.0,0.0,Neutral


In [28]:
ethereum_sentiment.to_csv(join(DATA_DIR, "ethereum_sentiment.csv"))
ethereum_sentiment.head()

,tweet_id,name,verified,created_at,retweet_count,favorite_count,followers_count,hashtags,text,clean_text,textblob_sentiment,textblob_summary,vader_sentiment_pos,vader_sentiment_neg,vader_sentiment_neu,vader_sentiment_compound,vader_summary
15,9,The Moon,True,2022-03-05 16:55:01+00:00,266,1776,1013020,"[{'text': 'crypto', 'indices': [12, 19]}]",Choose your #crypto team:\n\nTeam 🅰️ ...,[choose your #crypto team\n\nteam 🅰️ ...,0.0,Neutral,0.0,0.000,1.000,0.000,Neutral
34,9,The Moon,True,2022-03-05 16:55:01+00:00,266,1776,1013020,"[{'text': 'crypto', 'indices': [12, 19]}]",Choose your #crypto team:\n\nTeam 🅰️ ...,[choose your #crypto team\n\nteam 🅰️ ...,0.0,Neutral,0.0,0.000,1.000,0.000,Neutral
57,9,The Moon,True,2022-03-05 16:55:01+00:00,266,1776,1013020,"[{'text': 'crypto', 'indices': [12, 19]}]",Choose your #crypto team:\n\nTeam 🅰️ ...,[choose your #crypto team\n\nteam 🅰️ ...,0.0,Neutral,0.0,0.000,1.000,0.000,Neutral
72,24,OKX_Brasil (antiga OKEx),True,2022-03-05 00:00:58+00:00,17,54,4541,"[{'text': 'BTC', 'indices': [52, 56]}, {'text'...",Top 10 moedas mencionadas no Twitter esta sema...,[top 10 moedas mencionadas no twitter esta sem...,0.5,Positive,0.0,0.136,0.864,-0.296,Negative
82,9,The Moon,True,2022-03-05 16:55:01+00:00,266,1776,1013020,"[{'text': 'crypto', 'indices': [12, 19]}]",Choose your #crypto team:\n\nTeam 🅰️ ...,[choose your #crypto team\n\nteam 🅰️ ...,0.0,Neutral,0.0,0.000,1.000,0.000,Neutral


In [29]:
doge_sentiment.to_csv(join(DATA_DIR, "doge_sentiment.csv"))
doge_sentiment.head()

,tweet_id,name,verified,created_at,retweet_count,favorite_count,followers_count,hashtags,text,clean_text,textblob_sentiment,textblob_summary,vader_sentiment_pos,vader_sentiment_neg,vader_sentiment_neu,vader_sentiment_compound,vader_summary
15,9,The Moon,True,2022-03-05 16:55:01+00:00,266,1776,1013020,"[{'text': 'crypto', 'indices': [12, 19]}]",Choose your #crypto team:\n\nTeam 🅰️ ...,[choose your #crypto team\n\nteam 🅰️ ...,0.0,Neutral,0.0,0.0,1.0,0.0,Neutral
20,14,David Gokhshtein,True,2022-03-05 13:43:06+00:00,148,1536,610961,"[{'text': 'crypto', 'indices': [26, 33]}]",How many people came into #crypto because of $...,[how many people came into #crypto because of ...,0.5,Positive,0.0,0.0,1.0,0.0,Neutral
34,9,The Moon,True,2022-03-05 16:55:01+00:00,266,1776,1013020,"[{'text': 'crypto', 'indices': [12, 19]}]",Choose your #crypto team:\n\nTeam 🅰️ ...,[choose your #crypto team\n\nteam 🅰️ ...,0.0,Neutral,0.0,0.0,1.0,0.0,Neutral
39,14,David Gokhshtein,True,2022-03-05 13:43:06+00:00,148,1536,610961,"[{'text': 'crypto', 'indices': [26, 33]}]",How many people came into #crypto because of $...,[how many people came into #crypto because of ...,0.5,Positive,0.0,0.0,1.0,0.0,Neutral
57,9,The Moon,True,2022-03-05 16:55:01+00:00,266,1776,1013020,"[{'text': 'crypto', 'indices': [12, 19]}]",Choose your #crypto team:\n\nTeam 🅰️ ...,[choose your #crypto team\n\nteam 🅰️ ...,0.0,Neutral,0.0,0.0,1.0,0.0,Neutral


In [30]:
solana_sentiment.to_csv(join(DATA_DIR, "solana_sentiment.csv"))
solana_sentiment.head()

,tweet_id,name,verified,created_at,retweet_count,favorite_count,followers_count,hashtags,text,clean_text,textblob_sentiment,textblob_summary,vader_sentiment_pos,vader_sentiment_neg,vader_sentiment_neu,vader_sentiment_compound,vader_summary
499,12,The Moon,True,2022-03-06 19:09:34+00:00,127,897,1013020,"[{'text': 'crypto', 'indices': [7, 14]}]",Pick a #crypto team:\n\nTeam 🅰️ T...,[pick a #crypto team\n\nteam 🅰️ t...,0.0,Neutral,0.0,0.0,1.0,0.0,Neutral
514,12,The Moon,True,2022-03-06 19:09:34+00:00,127,897,1013020,"[{'text': 'crypto', 'indices': [7, 14]}]",Pick a #crypto team:\n\nTeam 🅰️ T...,[pick a #crypto team\n\nteam 🅰️ t...,0.0,Neutral,0.0,0.0,1.0,0.0,Neutral
533,12,The Moon,True,2022-03-06 19:09:34+00:00,127,897,1013020,"[{'text': 'crypto', 'indices': [7, 14]}]",Pick a #crypto team:\n\nTeam 🅰️ T...,[pick a #crypto team\n\nteam 🅰️ t...,0.0,Neutral,0.0,0.0,1.0,0.0,Neutral
553,12,The Moon,True,2022-03-06 19:09:34+00:00,127,897,1013020,"[{'text': 'crypto', 'indices': [7, 14]}]",Pick a #crypto team:\n\nTeam 🅰️ T...,[pick a #crypto team\n\nteam 🅰️ t...,0.0,Neutral,0.0,0.0,1.0,0.0,Neutral
587,12,The Moon,True,2022-03-06 19:09:34+00:00,127,897,1013020,"[{'text': 'crypto', 'indices': [7, 14]}]",Pick a #crypto team:\n\nTeam 🅰️ T...,[pick a #crypto team\n\nteam 🅰️ t...,0.0,Neutral,0.0,0.0,1.0,0.0,Neutral


In [31]:
avalanche_sentiment.to_csv(join(DATA_DIR, "avalanche_sentiment.csv"))
avalanche_sentiment.head()

,tweet_id,name,verified,created_at,retweet_count,favorite_count,followers_count,hashtags,text,clean_text,textblob_sentiment,textblob_summary,vader_sentiment_pos,vader_sentiment_neg,vader_sentiment_neu,vader_sentiment_compound,vader_summary
146,37,Túlio,True,2022-03-05 13:35:23+00:00,96,134,840626,"[{'text': 'altcoins', 'indices': [17, 26]}, {'...",Looking for some #altcoins that will skyrocket...,[looking for some #altcoins that will skyrocke...,0.0,Neutral,0.0,0.0,1.0,0.0,Neutral
187,37,Túlio,True,2022-03-05 13:35:23+00:00,96,134,840626,"[{'text': 'altcoins', 'indices': [17, 26]}, {'...",Looking for some #altcoins that will skyrocket...,[looking for some #altcoins that will skyrocke...,0.0,Neutral,0.0,0.0,1.0,0.0,Neutral
236,37,Túlio,True,2022-03-05 13:35:23+00:00,96,134,840626,"[{'text': 'altcoins', 'indices': [17, 26]}, {'...",Looking for some #altcoins that will skyrocket...,[looking for some #altcoins that will skyrocke...,0.0,Neutral,0.0,0.0,1.0,0.0,Neutral
250,51,Túlio,True,2022-03-05 13:35:23+00:00,96,134,840626,"[{'text': 'altcoins', 'indices': [17, 26]}, {'...",Looking for some #altcoins that will skyrocket...,[looking for some #altcoins that will skyrocke...,0.0,Neutral,0.0,0.0,1.0,0.0,Neutral
291,37,Túlio,True,2022-03-05 13:35:23+00:00,96,134,840626,"[{'text': 'altcoins', 'indices': [17, 26]}, {'...",Looking for some #altcoins that will skyrocket...,[looking for some #altcoins that will skyrocke...,0.0,Neutral,0.0,0.0,1.0,0.0,Neutral


In [32]:
crypto_sentiment.to_csv(join(DATA_DIR, "crypto_sentiment.csv"))
crypto_sentiment.head()

,tweet_id,name,verified,created_at,retweet_count,favorite_count,followers_count,hashtags,text,clean_text,textblob_sentiment,textblob_summary,vader_sentiment_pos,vader_sentiment_neg,vader_sentiment_neu,vader_sentiment_compound,vader_summary
0,0,Bitcoin News,True,2022-03-05 09:33:07+00:00,151,520,2437101,"[{'text': 'crypto', 'indices': [61, 68]}]","""It is an open ledger, trying to sneak lots of...",[it is an open ledger trying to sneak lots of ...,0.25,Positive,0.0,0.000,1.000,0.0000,Neutral
1,1,Bitcoin News,True,2022-03-05 12:03:14+00:00,87,245,2437101,"[{'text': 'crypto', 'indices': [5, 12]}]",“The #crypto market today has a market capital...,[“the #crypto market today has a market capita...,-0.20,Negative,0.0,0.000,1.000,0.0000,Neutral
2,2,Bitcoin News,True,2022-03-05 13:33:29+00:00,95,245,2437101,"[{'text': 'cryptocurrency', 'indices': [88, 10...",G7 countries and the EU are looking at ways to...,[g7 countries and the eu are looking at ways t...,0.00,Neutral,0.0,0.109,0.891,-0.2960,Negative
3,3,Watcher.Guru,True,2022-03-05 11:01:13+00:00,149,1043,708057,"[{'text': 'cryptocurrency', 'indices': [77, 92]}]",JUST IN: 🇸🇬 Singapore has introduced sanctions...,[just in 🇸🇬 singapore has introduced sanctions...,0.00,Neutral,0.0,0.167,0.833,-0.5574,Negative
5,5,Ian Heinisch,True,2022-03-05 21:34:37+00:00,16,27,7822,"[{'text': 'cryptocurrency', 'indices': [54, 69]}]",I can not wait for the move to earn app. @Yeti...,[i can not wait for the move to earn app @yeti...,0.00,Neutral,0.0,0.000,1.000,0.0000,Neutral


In [33]:
len_doge = len(doge_sentiment)
len_btc = len(bitcoin_sentiment)
len_avax = len(avalanche_sentiment)
len_eth = len(ethereum_sentiment)
len_sol = len(solana_sentiment)

print(len(text_data) - len_doge - len_sol - len_eth - len_btc - len_avax)
print(
    len(text_data)
    - len_doge
    - len_sol
    - len_eth
    - len_btc
    - len_avax
    - len(crypto_sentiment)
)

2750
369


Not all the tweets pertain to the coins - there is clearly some error within the API as there are some tweets with no hashtags or mention of the specified criteria. Perhaps it pulled in some viral tweets that were important across all spaces despite not explicitly mentioning the keywords.